In [ ]:
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px

from eda_utils import (
    read_scip_log,
    format_log_df,
    operations_plots,
    objective_function_plots,
    get_month_difference,
    get_revenue_before_eow,
    get_business_sales,
)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

%load_ext autoreload
%autoreload 2


In [ ]:
business_sales_path = "../data/business_sales.csv"

with open("../lp_logs/experiments_results.json") as json_file:
    experiment_results = json.load(json_file)
with open("../lp_logs/params.json") as json_file:
    PARAMS = json.load(json_file)

In [ ]:
for experiment, value in experiment_results.items():
    Log_df = read_scip_log(f"../lp_logs/{experiment}.log")
    Log_df = format_log_df(Log_df)

    # periods_before_eow = get_month_difference(
    #    experiment_results[experiment]["fecha_inicio"],
    #    experiment_results[experiment]["fecha_fin_ejercicio"],
    # )

    periods_before_eow = experiment_results[experiment].get(
        "eow", experiment_results[experiment]["periodos_modelo"]
    )
    experiment_results[experiment]["periods_before_eow"] = periods_before_eow

    (
        experiment_results[experiment]["lp_sum_cost_function"],
        experiment_results[experiment]["lp_stockquote_before_eow_SELL"],
        experiment_results[experiment]["stock_eow_df"],
    ) = get_revenue_before_eow(
        Log_df, periods_before_eow, PARAMS, experiment_results, experiment
    )

In [ ]:
df_table = pd.DataFrame()
for experiment, value in experiment_results.items():
    series = (
        pd.Series(
            {
                "experiment": experiment,
                "fecha_inicio": value["fecha_inicio"],
                "fecha_fin_ejercicio": value["fecha_fin_ejercicio"],
                "periodos_modelo": value["periodos_modelo"],
                "fix_prices": value["fix_prices"],
                "lp_stock_historical_cost": value["lp_stock_history_cost"],
                "lp_sum_cost_function": value["lp_sum_cost_function"],
                "lp_stockquote_before_eow_SELL": value["lp_stockquote_before_eow_SELL"],
                "lp_grand_total_before_eow": value["lp_sum_cost_function"]
                - value["lp_stock_history_cost"]
                + value["lp_stockquote_before_eow_SELL"],
                "business_grand_total": value["business_results"]["grand_total"],
                "business_sales_margin": value["business_results"]["sales_margin"],
                "stock_revenue_sum": value["business_results"]["stock_revenue_sum"],
                "stock_cost_sum": value["business_results"]["stock_cost_sum"],
            }
        )
        .to_frame()
        .T
    )
    df_table = pd.concat([df_table, series])

df_table["impact"] = (
    df_table["lp_grand_total_before_eow"] - df_table["business_grand_total"]
) / df_table["business_grand_total"]
df_table["impact"] = (df_table["impact"].apply(round, args=[2]) * 100).astype(int)


df_table.loc[df_table.experiment == "2019_24periods_real_prices"]

In [ ]:
experiment_results["2019_24periods_real_prices"]

In [ ]:
business_final_stock = pd.read_csv(
    "../lp_logs/df_final_stock_value_2019_48period_fix_prices.csv"
)
business_final_stock = business_final_stock.loc[business_final_stock["qty"] != 0]
business_final_stock

In [ ]:
pd.set_option("display.max_rows", None)

df = pd.read_csv("../ventastest.csv")
df.loc[df.VENTA_VALOR > 0].sum()

In [ ]:
experiment = "2019_24periods_real_prices"
# periods_before_eow = experiment_results[experiment]["periods_before_eow"]

Log_df = read_scip_log(f"../lp_logs/{experiment}.log")
Log_df = format_log_df(Log_df)

operations_plots(
    Log_df,
    periodos_modelo=experiment_results[experiment]["periodos_modelo"],
    periods_before_eow=periods_before_eow,
)

objective_function_plots(
    Log_df,
    periodos_modelo=experiment_results[experiment]["periodos_modelo"],
    periods_before_eow=periods_before_eow,
)

get_business_sales(business_sales_path, experiment, experiment_results)

In [ ]:
print(
    Log_df.loc[Log_df["var"] == "x"]
    .loc[Log_df["t"] == 1]
    .loc[Log_df["class"] == "3"]["value"]
    .sum(),
    Log_df.loc[Log_df["var"] == "x"]
    .loc[Log_df["t"] == Log_df["t"].max()]
    .loc[Log_df["class"] == "3"]["value"]
    .sum(),
)

In [ ]:
df_p_fix = pd.read_csv("../lp_logs/df_precios_fixFalse.csv", index_col=False)
df_p_fix["YYYYMM"] = pd.to_datetime(df_p_fix["YYYYMM"], format="%Y%m")
df_p_fix = df_p_fix.melt(
    id_vars=["YYYYMM"],
    value_vars=["VAQUILLONAS270", "VAQUILLONAS391", "NOVILLITOS300", "NOVILLITOS391"],
)

fig0 = sns.lineplot(data=df_p_fix, x="YYYYMM", y="value", hue="variable")
fig0.figure.set_size_inches(14, 6)